In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib import axes
from skimage.feature import graycomatrix, graycoprops
from PIL import Image

validate the new features on simple black and white images

In [ ]:
s_image = np.array([0,1,0,1,0,1,0,1] * 8)
s_image = s_image.reshape((8, 8))

plt.imshow(s_image, cmap='gray')

GLCMs

In [ ]:
glcm0 =graycomatrix(s_image, distances=[1], angles=[0], levels=2, symmetric=False)
glcm45 = graycomatrix(s_image, distances=[1], angles=[np.pi/4], levels=2, symmetric=False)
glcm90 = graycomatrix(s_image, distances=[1], angles=[np.pi/2], levels=2, symmetric=False)
glcm135 = graycomatrix(s_image, distances=[1], angles=[3 * np.pi/4], levels=2, symmetric=False)

glcm = (glcm0 + glcm45 + glcm90 + glcm135)/4
glcm

## Functions for computing feature

In [ ]:
contrast_true = graycoprops(glcm, 'contrast')
# graycoprops(glcm, 'correlation')
# graycoprops(glcm, 'homogeneity')
# graycoprops(glcm, 'energy')

def normalize_glcm(mean_glcm):
    normalized_glcm = mean_glcm / np.sum(mean_glcm)

    return normalized_glcm

def contrast_man(norm_glcm):
    contr = 0.0
    Ng = len(norm_glcm)

    for i in range(0, Ng):
        for j in range(0, Ng):
            contr += norm_glcm[i,j] * (i-j)**2

    return contr

normed_glcm = normalize_glcm(glcm)
contrast_man(normed_glcm) == contrast_true
normed_glcm


In [ ]:
def correlation_man(norm_glcm):
    Ng = norm_glcm.shape[0]
    indices = np.arange(Ng) + 1
    # i, j = np.ogrid[0:Ng, 0:Ng] 
    i, j = np.ogrid[1:Ng+1, 1:Ng+1]

    px = np.sum(norm_glcm, axis=1)
    px = px.squeeze()
    py = np.sum(norm_glcm, axis=0)
    py = py.squeeze()

    mu_x = np.dot(indices, px)
    mu_y = np.dot(indices, py)
    
    # sigma_x = np.sum((i[0, :] - mu_x)**2 * px)
    # sigma_y = np.sum((j[0, :] - mu_y)**2 * py)
    sigma_x = np.sqrt(np.sum((i[0, :] - mu_x)**2 * px))
    sigma_y = np.sqrt(np.sum((i[0, :] - mu_y)**2 * py))

    # i_j_p = i * j * norm_glcm
    i_j_p = np.dot(i*j, norm_glcm.squeeze())
    sum_i_j_p = np.sum(i_j_p)

    numerator = sum_i_j_p - (mu_x * mu_y)
    denominator = sigma_x * sigma_y
    
    return numerator / denominator


if correlation_man(normed_glcm) == graycoprops(glcm, 'correlation'):
    print("match!")
else: print(correlation_man(normed_glcm) - graycoprops(glcm, 'correlation'))

In [ ]:
def homogeneity_man(norm_glcm):
    homog = 0.0
    Ng = len(norm_glcm)

    for i in range(0, Ng):
        for j in range(0, Ng):
            homog += norm_glcm[i, j] / (1 + (i - j)**2)

    return homog

homogeneity_man(normed_glcm) == graycoprops(glcm, 'homogeneity')

In [ ]:
def energy_man(norm_glcm):
    return np.sqrt(np.sum(norm_glcm**2))

energy_man(normed_glcm) == graycoprops(glcm, 'energy')

# Pipeline
GS —> GLCM (mean) \
&ensp; Number of color levels: size of (square) matrix N \
&ensp; Enlist GLCMs of all directions (this can help when tackling rotation invariance)

GLCM —> Normed GLCM \
Normalize GLCM.
ALL first order statistics needed for computing texture features
* mu_x
* mu_y
* sigma_x
* sigma_y
* ...

normed GLCM —> All features \
Compute and Enlist features (Contrast, Energy, Homogeneity, Correlation, Autocorrelation, Cluster prominence, Cluster shade, Difference entropy, Difference variance, Dissimilarity, Entropy, Information measure of correlation 1, Information measure of correlation 2, Inverse difference, Sum average, Sum entropy, Sum of squares, Sum variance) from (Löfstedt et al. 2019).

**Following structure of step 4 in the paper with a Psi component and a Phi component for the function of each feature.**

Refference: Tommy Löfstedt, Patrik Brynolfsson , Thomas Asklund, Tufve Nyholm, Anders Garpebring. "Gray-level invariant Haralick texture features" (2019)

## New features

- autocorrelation
- cluster prominence
- cluster shade
- dissimilarity
- entropy
- difference entropy
- difference variance
- inverse difference
- sum average
- sum of squares
- sum variance

In [ ]:
import numpy as np
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib import axes
from skimage.feature import graycomatrix, graycomatrix as glcm_compute
from scipy.stats import entropy as scipy_entropy

In [ ]:
# PRE: Image to Grayscale 
s_image = np.array([0,1,0,1,0,1,0,1] * 8)
s_image = s_image.reshape((8, 8))

In [ ]:
# STEP 1: Grayscale to GLCM

def compute_glcms(gs_image, levels = 2):

    glcm0 = graycomatrix(gs_image, distances=[1], angles=[0], levels=levels, symmetric=False)
    glcm45 = graycomatrix(gs_image, distances=[1], angles=[np.pi/4], levels=levels, symmetric=False)
    glcm90 = graycomatrix(gs_image, distances=[1], angles=[np.pi/2], levels=levels, symmetric=False)
    glcm135 = graycomatrix(gs_image, distances=[1], angles=[3 * np.pi/4], levels=levels, symmetric=False)
    mean_glcm = (glcm0 + glcm45 + glcm90 + glcm135)/4

    glcms = {'mean': mean_glcm, '0': glcm0, '45': glcm45, '90': glcm90, '135': glcm135}

    return glcms

In [ ]:
# STEP 2: Normalize GLCM and Compute First-Order Statistics

def pre_feature_statistics(mean_glcm):
    """
    Returns: 
    stats : dict
        Dictionary containing normed_glcm, mu_x, mu_y, sigma_x, sigma_y, px, py
    """

    normed_glcm = mean_glcm / np.sum(mean_glcm)
    N = normed_glcm.shape[0]
    i_indices = np.arange(N) + 1
    j_indices = np.arange(N) + 1
    
    # Marginal probabilities
    p_x = np.sum(normed_glcm, axis=1)  # Sum over j
    p_x = p_x.squeeze()
    p_y = np.sum(normed_glcm, axis=0)  # Sum over i
    p_y = p_y.squeeze()
    
    # Mean values
    indices = np.arange(N) + 1
    mu_x = np.dot(indices, p_x)
    mu_y = np.dot(indices, p_y)
    
    # Standard deviations
    sigma_x = np.sqrt(np.sum(((i_indices - mu_x) ** 2) * p_x))
    sigma_y = np.sqrt(np.sum(((j_indices - mu_y) ** 2) * p_y))
    
    # HX and HY
    # p_x_nonzero = p_x[p_x > 0]
    # HX = -np.sum(p_x_nonzero * np.log2(p_x_nonzero))
    
    # p_y_nonzero = p_y[p_y > 0]
    # HY = -np.sum(p_y_nonzero * np.log2(p_y_nonzero))
    
    # Compute p_x+y and p_x-y for later use
    # p_sum = np.zeros(2 * N - 1)
    # p_diff = np.zeros(N)
    
    # for i in range(N):
    #     for j in range(N):
    #         # p_x+y
    #         p_sum[i + j] += normed_glcm[i, j]
    #         # p_x-y
    #         p_diff[abs(i - j)] += normed_glcm[i, j]
    
    stats = {
        'Pij' : normed_glcm,
        'mu_x': mu_x,
        'mu_y': mu_y,
        'sigma_x': sigma_x,
        'sigma_y': sigma_y,
        'p_x': p_x,
        'p_y': p_y,
        # 'HX': HX,
        # 'HY': HY,
        # 'p_sum': p_sum,
        # 'p_diff': p_diff
    }
    
    return stats


In [ ]:
s_glcms = compute_glcms(s_image)['mean']
pre_feature_statistics(s_glcms)

In [ ]:
# STEP 3: Compute Haralick Texture Features 
# Structure: Each feature has Psi and Phi components

def compute_contrast(stats):

    return 


def compute_energy(stats):

    return 


def compute_homogeneity(stats):
    
    return 


def compute_correlation(stats):
    
    return 

In [ ]:
def compute_autocorrelation(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]

    autocorrelation = 0.0

    for i in range(N):
        for j in range(N):
            i_1 = i + 1
            j_1 = j + 1

            # φ(i,j,g(P)) = i·j
            phi = i_1 * j_1

            # ψ(p(i,j)) = p(i,j)
            psi = normed_glcm[i,j]

            autocorrelation += phi * psi

    return autocorrelation


def compute_cluster_prominence(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    p_x = stats['p_x']

    # mu = sum(i * px_i)           ---***---
    # OR mu = mu_x + mu_y          ---***---
    mu =  np.dot(np.arange(1,N+1) * p_x)

    cluster_prominence = 0.0

    for i in range(N):
        for j in range(N):
            i_1 = i + 1
            j_1 = j + 1
            
            # φ(i,j,g(P)) = (i + j - 2*mu)^3 
            phi = (i_1 + j_1 - 2 * mu)**3

            # ψ(p(i,j)) = p(i,j)
            psi = normed_glcm[i,j]
            
            cluster_prominence += phi * psi
            
    return cluster_prominence


def compute_cluster_shade(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    p_x = stats['p_x']

    # mu = sum(i * px_i)           ---***---
    # OR mu = mu_x + mu_y          ---***---
    mu =  np.dot(np.arange(1,N+1) * p_x)

    cluster_shade = 0.0

    for i in range(N):
        for j in range(N):
            i_1 = i + 1
            j_1 = j + 1
            
            # φ(i,j,g(P)) = (i + j - 2*mu)^4 
            phi = (i_1 + j_1 - 2 * mu)**4

            # ψ(p(i,j)) = p(i,j)
            psi = normed_glcm[i,j]
            
            cluster_shade += phi * psi
    
    return cluster_shade


def compute_dissimilarity(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    
    dissimilarity = 0.0
    
    for i in range(N):
        for j in range(N):
            i_1 = i + 1
            j_1 = j + 1
            
            # φ(i,j,g(P)) = |i - j|
            phi = abs(i_1 - j_1)
            
            # ψ(p(i,j)) = p(i,j)
            psi = normed_glcm[i, j] 
            
            dissimilarity += phi * psi
    
    return dissimilarity


def compute_entropy(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    
    entropy_val = 0.0
    
    for i in range(N):
        for j in range(N):
            p = normed_glcm[i, j]
            if p > 0:
                # φ(i,j,g(P)) = 1
                phi = 1
                
                # ψ(p(i,j)) = -p(i,j) * log(p(i,j))
                psi = -p * np.log(p)
                
                entropy_val += phi * psi
    
    return entropy_val

**Questions**:
- Check entropy 
- What is the 2 mu in cluster prominence and cluster shade?
- Check HX and HY

In [ ]:
def compute_difference_entropy(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    
    # p_x-y(k)
    p_diff = np.zeros(N)
    
    for i in range(N):
        for j in range(N):
            k = abs((i + 1) - (j + 1))
            if k < N:
                p_diff[k] += normed_glcm[i, j]
    

    diff_entropy = 0.0
    for k in range(N):
        if p_diff[k] > 0:
            diff_entropy -= p_diff[k] * np.log(p_diff[k])
    
    return diff_entropy


def compute_difference_variance(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    
    return 


def compute_inverse_difference(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    
    return 


def compute_sum_average(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    
    return 


def compute_sum_entropy(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    
    return


def compute_sum_of_squares(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]

    return 


def compute_sum_variance(stats):
    normed_glcm = stats['Pij']
    N = normed_glcm.shape[0]
    
    return 


def compute_information_measures_of_correlation(stats):

    return 

### Composition  (normalized glcm --> All features)

In [ ]:
def compute_all(mean_glcm):

    # contrast = 
    #...

    return 